<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/vector_embeddings/examples_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ollama langchain_community --quiet
!pip install openai --quiet
!pip install faiss-cpu --quiet

host="localhost:11434"
modelid="chevalblanc/gpt-4o-mini"
embedding_model = "mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")



In [ ]:
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {embedding_model}")


In [ ]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [ ]:
def embed(text: str) -> list[float]:
  return llm.embeddings.create(input = [text], model=embedding_model).data[0].embedding

def createEmbeddings(texts: list[str]) -> list[list[float]]:
  return [embed(txt) for txt in texts]


Prompt to create demo data:

your task is to create 10 demo user for a app.

output the users in JSON format.

Every user should have
* username
* age
* interests for investment category (TECHNOLOGY, AI, CHEMICAL, HEALTCARE)
* risk level high, medium, low
* personal interests and hobbies

In [ ]:


demo_users = [
    {
      "username": "techsavvy_sarah",
      "age": 28,
      "investment_interests": ["TECHNOLOGY", "AI"],
      "risk_level": "high",
      "personal_interests": ["coding", "gaming", "cryptocurrency", "VR technology", "startup podcasts"]
    },
    {
      "username": "conservative_mike",
      "age": 45,
      "investment_interests": ["HEALTHCARE", "CHEMICAL"],
      "risk_level": "low",
      "personal_interests": ["golf", "reading financial news", "gardening", "wine collecting", "family time"]
    },
    {
      "username": "ai_enthusiast_emma",
      "age": 32,
      "investment_interests": ["AI", "TECHNOLOGY"],
      "risk_level": "high",
      "personal_interests": ["machine learning courses", "robotics", "sci-fi movies", "chess", "tech conferences"]
    },
    {
      "username": "balanced_ben",
      "age": 38,
      "investment_interests": ["HEALTHCARE", "TECHNOLOGY", "CHEMICAL"],
      "risk_level": "medium",
      "personal_interests": ["hiking", "photography", "cooking", "travel", "fitness tracking"]
    },
    {
      "username": "pharma_investor_lisa",
      "age": 41,
      "investment_interests": ["HEALTHCARE", "CHEMICAL"],
      "risk_level": "medium",
      "personal_interests": ["medical research reading", "yoga", "volunteer work", "painting", "classical music"]
    },
    {
      "username": "young_hustler_alex",
      "age": 24,
      "investment_interests": ["AI", "TECHNOLOGY"],
      "risk_level": "high",
      "personal_interests": ["day trading", "social media", "electric vehicles", "music production", "entrepreneurship"]
    },
    {
      "username": "steady_investor_carol",
      "age": 52,
      "investment_interests": ["HEALTHCARE"],
      "risk_level": "low",
      "personal_interests": ["book club", "knitting", "bird watching", "retirement planning", "grandchildren"]
    },
    {
      "username": "diversified_david",
      "age": 35,
      "investment_interests": ["TECHNOLOGY", "HEALTHCARE", "AI", "CHEMICAL"],
      "risk_level": "medium",
      "personal_interests": ["marathon running", "financial podcasts", "home brewing", "board games", "sustainable living"]
    },
    {
      "username": "biotech_researcher_nina",
      "age": 29,
      "investment_interests": ["HEALTHCARE", "CHEMICAL", "AI"],
      "risk_level": "medium",
      "personal_interests": ["laboratory work", "scientific journals", "rock climbing", "environmental activism", "documentary films"]
    },
    {
      "username": "risk_taker_ryan",
      "age": 26,
      "investment_interests": ["AI", "TECHNOLOGY"],
      "risk_level": "high",
      "personal_interests": ["extreme sports", "crypto trading", "gaming livestreaming", "motorcycle riding", "tech startup networking"]
    }
  ]


In [ ]:
import faiss
import numpy as np


def userdescription(user):
  interests=join(user['personal_interests'],f'\n -')
  return f"User {user['username']} is {user['age']} years old. interests: {interests}\n\n\"

demo_profiles=[userdescription(user) for user in demo_users]

# Convert to embeddings
embeddings = createEmbeddings(demo_profiles)

# Store in FAISS index for similarity search
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [ ]:
# Query the most similar sentence
query = embed("AI evolves with data.")
D, I = index.search(np.array([query]), 1)
print(f"The most similar sentence is: {sentences[I[0][0]]}")

The most similar sentence is: AI requires large datasets to train models.


In [ ]:
from pydantic import BaseModel

def call(system_prompt: str, message: str, output_schema: BaseModel | None = None , model: str = modelid) -> str:

  messages=[
          {
              "role": "user",
              "content": message,
          },]
  completion = llm.chat.completions.parse(
      model=modelid,
      messages=[ {"role": "system", "content": system_prompt},]
      +messages,
      temperature=0.0,
   #   max_tokens=4096,
      response_format=output_schema
  )

  return completion.choices[0].message.parsed

NameError: name 'BaseModel' is not defined